In [ ]:
import pandas as pd
import numpy as np

from scipy.signal import argrelextrema

from scipy.misc import imread
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
locations = pd.read_hdf("data/backup/complete_paths.h5", "data")
locations = locations[((locations['ice'] > 0) & (locations['ice'] <= 100)) | (locations["ice"] == 255)]

vectorspace = np.fromfile("data/backup/complete_vector.pyarray").reshape((12454,255))
locations = locations[np.all(vectorspace > 0, axis=1)]
vectorspace = vectorspace[np.all(vectorspace > 0, axis=1)]

In [ ]:
locations = pd.read_hdf("data/backup/cleaned_cluster.h5", "data")

In [ ]:
def build_vs(path):
    path = "/media/lucas/data/images/" + path.split("/")[-1]
    
    try:
        im = imread(path, mode="L")
    except FileNotFoundError:
        return np.zeros(255)
    
    if np.mean(im) < 50:
        return np.zeros(255)

    hist = np.histogram(im, bins=np.arange(0, 256), normed=True)[0]
    
    return hist

In [ ]:
vectorspace = locations["image"].apply(build_vs)

a = vectorspace.values.copy()
vectorspace = np.ndarray((12454,255))

i = 0
for val in a:
    vectorspace[i] = a[i]
    
    i += 1

vectorspace.tofile("data/backup/complete_vector.pyarray")

In [ ]:
vec = np.array(list(map(lambda x: np.convolve(x, np.ones((50,))/50, mode='valid'), vectorspace)))

In [ ]:
#km = DBSCAN(eps=0.05, min_samples=100)
km = KMeans(n_clusters=16)
clusters = km.fit_predict(vec)

In [ ]:
pca = PCA(n_components=2, svd_solver="randomized")
redux = pca.fit_transform(vec)

In [ ]:
cm = plt.cm.get_cmap('RdYlBu')
plt.scatter(redux[:, 0], redux[:, 1], c=clusters, cmap=cm)
plt.colorbar()

In [ ]:
locations["cluster"] = clusters
locations.to_hdf("data/backup/cleaned_cluster.h5", "data")

In [ ]:
im = imread("/media/lucas/data/images/20150718-1001.jpeg", mode="L")[375:800, :]

In [ ]:
hist = np.histogram(im, bins=np.arange(0, 256), normed=True)

In [ ]:
a = np.convolve(hist[0], np.ones((20,))/20, mode='valid')
argrelextrema(a, np.greater)[0]

In [ ]:
plt.plot(hist[1][:-1], hist[0], lw=2) 

In [ ]:
locations[locations.index.strftime("%Y%m%d") == "20150719"]